# Insect Hopper IL Pipeline

In [34]:
import scipy.io
import torch.nn as nn
import torch.optim as optim
import os
import mujoco
import mujoco.viewer
import dm_control
import mediapy as media

In [35]:
model = mujoco.MjModel.from_xml_path("hopper.xml")
# model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

viewer = mujoco.viewer

viewer.launch(model, data)


2025-08-23 16:19:04.103 python[5331:17780922] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


In [27]:
class JumpingData:
    """
    ### JumpingData Fields
            `.Euler_XYZ`: the euler XYZ angles in *radians*.\n
            `.driving_signals`: control signals.\n
            `.torque`: the torque.\n
            `.position`: the position in *meters*.
    """
    def __init__(self, filename: str, mat_dict: dict):
        """
            ### Parameters
            `filename` is the name of the associated .mat file. This is used as an ID.\n
            `mat_dict` is the dictionary returned from extracting the .mat data
        """
        self.name = filename
        self.dict = mat_dict

        # Shortcuts to what we care about
        self.Euler_XYZ = self.dict["rst_Eul_XYZ"][0][0][0] # Radians
        self.driving_signals = self.dict["rst_driving_signals"][0][0][0]
        self.torque = self.dict["rst_torque_b"][0][0][0]
        self.position = self.dict["rst_p_raw"][0][0][0] # meters
        self.thrust = self.dict["rst_thrust"][0][0][0] # milligrams-force


jumping_data_dir = "/Users/cassandrahe/MIT Dropbox/Cassandra He/jumping_data"
data = []

for folder in os.listdir(jumping_data_dir):
    if folder[0] == ".": continue # to get around whatever .DS_store files dropbox adds
    print(folder)
    for file in os.listdir(f"{jumping_data_dir}/{folder}"):
        print(file)
        # TODO: data processing here
        mat = scipy.io.loadmat(f"{jumping_data_dir}/{folder}/{file}")
        # print(mat.keys())
        data.append(JumpingData(filename=file, mat_dict=mat))


# mat = scipy.io.loadmat()#TODO: EXPERT DATA()
    
# print(mat.keys)
# euler_xyz = mat["EulerXYZ"]  # shape (T, 3) maybe
# torques = mat["torque"]      # shape (T, n)

7cm_setpoint
closedloop23_Nemo44_2024-03-29_19-37-34.mat
closedloop21_Nemo44_2024-03-29_19-20-55.mat
closedloop20_Nemo44_2024-03-29_19-12-37.mat
closedloop22_Nemo44_2024-03-29_19-32-03.mat
closedloop18_Nemo44_2024-03-29_18-54-23.mat
10cm_setpoint
closedloop9_Nemo44_2024-04-25_18-50-54.mat
closedloop10_Nemo44_2024-04-25_18-59-11.mat
closedloop8_Nemo44_2024-04-25_18-34-18.mat
closedloop6_Nemo44_2024-04-25_16-22-34.mat
closedloop12_Nemo44_2024-04-25_21-00-29.mat
closedloop7_Nemo44_2024-04-25_18-19-30.mat
closedloop11_Nemo44_2024-04-25_19-07-59.mat
4cm_setpoint
closedloop14_Nemo44_2024-04-21_19-28-04.mat


/Users/cassandrahe/anaconda3/envs/mujoco_il/lib/python3.11/site-packages/scipy/io/matlab/_mio.py:235: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Considerscipy.io.matlab.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


closedloop9_Nemo44_2024-04-21_18-48-33.mat
closedloop12_Nemo44_2024-04-21_19-12-22.mat
closedloop10_Nemo44_2024-04-21_18-56-45.mat
closedloop11_Nemo44_2024-04-21_19-02-17.mat


In [28]:
data_7cm_1 = data[0]
print(f"{data_7cm_1.name} positions: {data_7cm_1.position}")

closedloop23_Nemo44_2024-03-29_19-37-34.mat positions: [[0.00000e+00]
 [1.00000e-04]
 [2.00000e-04]
 ...
 [1.49998e+01]
 [1.49999e+01]
 [1.50000e+01]]


## Wrap Dataset for BC

In [ ]:
import torch
from torch.utils.data import Dataset

class HopperExpertDataset(Dataset):
    def __init__(self, matfile):
        mat = scipy.io.loadmat(matfile)
        self.obs = mat["EulerXYZ"]  # plus other signals (vel, etc.)
        self.acts = mat["torque"]   # actuator targets

    def __len__(self):
        return len(self.obs)

    def __getitem__(self, idx):
        obs = torch.tensor(self.obs[idx], dtype=torch.float32)
        act = torch.tensor(self.acts[idx], dtype=torch.float32)
        return obs, act


## Minial BC Loop

In [ ]:
policy = nn.Sequential(
    nn.Linear(obs_dim, 128),
    nn.ReLU(),
    nn.Linear(128, act_dim)
)

dataset = HopperExpertDataset("expert_data.mat")
loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

opt = optim.Adam(policy.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

for epoch in range(20):
    for obs, act in loader:
        pred = policy(obs)
        loss = loss_fn(pred, act)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(f"Epoch {epoch}: Loss={loss.item()}")